In [1]:
import pandas as pd
import numpy as np
import gymnasium as gym

In [2]:
df = pd.read_pickle("./data/raw/binance-BTCUSDT-5m.pkl")

In [ ]:
df.tail()

In [3]:
df["feature_close"] = df["close"]
df["feature_open"] = df["open"]
df["feature_high"] = df["high"]
df["feature_low"] = df["low"]
df["feature_volume"] = df["volume"]
df.dropna(inplace= True) # Clean your data !

In [ ]:
df

In [4]:
def reward_function(history):
    return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2]) #log (p_t / p_t-1 )

In [6]:
from gym_trading_env.environments import TradingEnv
env = gym.make(
        "TradingEnv",
        name= "BTCUSD",
        df = df,
        windows= 5,
        positions = [ -1, -0.5, 0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 'random', #Initial position
        trading_fees = 0.01/100, # 0.01% per stock buy / sell
        borrow_interest_rate= 0.0003/100, #per timestep (= 1h here)
        reward_function = reward_function,
        portfolio_initial_value = 1000, # in FIAT (here, USD)
        max_episode_duration = 500,
    )

In [10]:
import os
from common import SaveOnBestTrainingRewardCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir, exist_ok=True)

DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.